In [1]:
from astropy.table import Table, join, Column, vstack
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import fitsio as F
import desimodel.io
import desitarget.mtl
import desisim.quickcat
from astropy.io import fits
import json
import shutil
from desitarget.targetmask import desi_mask, obsconditions, bgs_mask
from collections import Counter

In [2]:
realn=0
fba_files = glob.glob("fba_bright_north_"+str(realn)+"/fba-*.fits")

In [3]:
tiles=[]
for i in fba_files:
    tiles.append(i.split('-')[1][-13:-5])
tiles=np.array(tiles)
tiles

array(['041812', '039964', '032754', ..., '041062', '046034', '036381'],
      dtype='<U6')

In [4]:
tile=tiles[0]
print("Testing with tile",tile)

Testing with tile 041812


In [5]:
target=F.read("targets_bright_north_"+str(realn)+".fits",columns=['TARGETID','RA','DEC','DESI_TARGET','BGS_TARGET','MWS_TARGET','SUBPRIORITY','PRIORITY_INIT','NUMOBS_INIT'])

In [6]:
assign=F.read("fba_bright_north_"+str(realn)+"/"+"fba-"+tile+".fits",
              columns=['FIBER','TARGETID','LOCATION','PETAL_LOC','TARGET_RA','TARGET_DEC','FA_TARGET','FA_TYPE'],ext="FASSIGN")
avail=F.read("fba_bright_north_"+str(realn)+"/"+"fba-"+tile+".fits",ext="FAVAIL")

In [7]:
len(assign), len(avail)

(5020, 42774)

In [8]:
assign

array([( 400, 288230378240522874,    0, 0, 128.3649244 , 73.56634431, 2305843009213693952, 1),
       ( 449, 288230378240522728,    1, 0, 128.388773  , 73.54570196, 2305843009213693952, 1),
       ( 420, 288230378240491583,    2, 0, 128.23088074, 73.56285095, 1152921504606846976, 1),
       ...,
       (4944, 288230378240508053, 9524, 9, 130.73736811, 72.30702013,         34359738368, 2),
       (4909, 288230378240536387, 9525, 9, 130.57627985, 72.28115657,          4294967296, 4),
       (4900, 288230378240507543, 9526, 9, 130.50802352, 72.28574327, 2305843017803628544, 3)],
      dtype=[('FIBER', '>i4'), ('TARGETID', '>i8'), ('LOCATION', '>i4'), ('PETAL_LOC', '>i2'), ('TARGET_RA', '>f8'), ('TARGET_DEC', '>f8'), ('FA_TARGET', '>i8'), ('FA_TYPE', 'u1')])

In [9]:
master=join(avail,assign,join_type='left',keys=["FIBER","LOCATION"])

In [10]:
master

LOCATION,FIBER,TARGETID_1,TARGETID_2,PETAL_LOC,TARGET_RA,TARGET_DEC,FA_TARGET,FA_TYPE
int32,int32,int64,int64,int16,float64,float64,int64,uint8
311,0,288230378240520452,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240490329,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240542339,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240520760,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240520628,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240542322,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240490296,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240542190,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1
311,0,288230378240520728,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1


In [11]:
dict1=dict(zip(target["TARGETID"],target["BGS_TARGET"]))

aval_bgs = [dict1.get(i,-1) for i in master["TARGETID_1"]]
master['BGS_TARGET_1']=aval_bgs
assi_bgs = [dict1.get(i,-1) for i in master["TARGETID_2"]]
master['BGS_TARGET_2']=assi_bgs

In [12]:
master

LOCATION,FIBER,TARGETID_1,TARGETID_2,PETAL_LOC,TARGET_RA,TARGET_DEC,FA_TARGET,FA_TYPE,BGS_TARGET_1,BGS_TARGET_2
int32,int32,int64,int64,int16,float64,float64,int64,uint8,int64,int64
311,0,288230378240520452,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1
311,0,288230378240490329,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,2,1
311,0,288230378240542339,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1
311,0,288230378240520760,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1
311,0,288230378240520628,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1
311,0,288230378240542322,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1
311,0,288230378240490296,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,1,1
311,0,288230378240542190,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1
311,0,288230378240520728,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1


In [13]:
is_availableintarget = np.zeros(len(master), dtype=bool)
id_availableintarget = np.isin(master['TARGETID_1'], target["TARGETID"])
is_availableintarget[id_availableintarget] = True
master['AVAIL_IN_TARGET'] = is_availableintarget

In [14]:
master

LOCATION,FIBER,TARGETID_1,TARGETID_2,PETAL_LOC,TARGET_RA,TARGET_DEC,FA_TARGET,FA_TYPE,BGS_TARGET_1,BGS_TARGET_2,AVAIL_IN_TARGET
int32,int32,int64,int64,int16,float64,float64,int64,uint8,int64,int64,bool
311,0,288230378240520452,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True
311,0,288230378240490329,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,2,1,True
311,0,288230378240542339,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1,False
311,0,288230378240520760,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True
311,0,288230378240520628,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True
311,0,288230378240542322,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1,False
311,0,288230378240490296,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,1,1,True
311,0,288230378240542190,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,-1,1,False
311,0,288230378240520728,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True


In [15]:
master_in=master[master["AVAIL_IN_TARGET"]==True]

In [16]:
master_in

LOCATION,FIBER,TARGETID_1,TARGETID_2,PETAL_LOC,TARGET_RA,TARGET_DEC,FA_TARGET,FA_TYPE,BGS_TARGET_1,BGS_TARGET_2,AVAIL_IN_TARGET
int32,int32,int64,int64,int16,float64,float64,int64,uint8,int64,int64,bool
311,0,288230378240520452,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True
311,0,288230378240490329,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,2,1,True
311,0,288230378240520760,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True
311,0,288230378240520628,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True
311,0,288230378240490296,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,1,1,True
311,0,288230378240520728,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,0,1,True
311,0,288230378240490459,288230378240490459,0,127.26505279541016,72.55120849609375,1152921504606846976,1,1,1,True
272,1,288230378240490328,288230378240490328,0,127.6200180053711,72.59854888916016,1152921504606846976,1,2,2,True
272,1,288230378240520759,288230378240490328,0,127.6200180053711,72.59854888916016,1152921504606846976,1,0,2,True


In [17]:
fiber=np.arange(0,5021,1)

In [18]:
fiber

array([   0,    1,    2, ..., 5018, 5019, 5020])

In [19]:
isbright_1=master_in["BGS_TARGET_1"]==2
isbright_2=master_in["BGS_TARGET_2"]==2
isfaint_1=master_in["BGS_TARGET_1"]==1
isfaint_2=master_in["BGS_TARGET_2"]==1
isfainthip_1=master_in["BGS_TARGET_1"]==9
isfainthip_2=master_in["BGS_TARGET_2"]==9
isstar_1=master_in["BGS_TARGET_1"]==0
isstar_2=master_in["BGS_TARGET_2"]==0

In [20]:
master_in[master_in["FIBER"]==2]

LOCATION,FIBER,TARGETID_1,TARGETID_2,PETAL_LOC,TARGET_RA,TARGET_DEC,FA_TARGET,FA_TYPE,BGS_TARGET_1,BGS_TARGET_2,AVAIL_IN_TARGET
int32,int32,int64,int64,int16,float64,float64,int64,uint8,int64,int64,bool
252,2,288230378240520812,288230378240520535,0,127.91436314702875,72.61715453557667,2305843009213693952,1,0,0,True
252,2,288230378240520608,288230378240520535,0,127.91436314702875,72.61715453557667,2305843009213693952,1,0,0,True
252,2,288230378240520535,288230378240520535,0,127.91436314702875,72.61715453557667,2305843009213693952,1,0,0,True
252,2,288230378240490330,288230378240520535,0,127.91436314702875,72.61715453557667,2305843009213693952,1,1,0,True
252,2,288230378240520749,288230378240520535,0,127.91436314702875,72.61715453557667,2305843009213693952,1,0,0,True
252,2,288230378240508279,288230378240520535,0,127.91436314702875,72.61715453557667,2305843009213693952,1,0,0,True


Looping for fibers in tile.

In [21]:
b2b,b2f,b2fh,b2s,f2b,f2f,f2fh,f2s,fh2b,fh2f,fh2fh,fh2s,s2b,s2f,s2fh,s2s,fib=[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
for i in range(5020):
    isfiber=master_in["FIBER"]==i
    b2b0=len(master_in[isfiber&isbright_1&isbright_2])
    b2f0=len(master_in[isfiber&isbright_1&isfaint_2])
    b2fh0=len(master_in[isfiber&isbright_1&isfainthip_2])
    b2s0=len(master_in[isfiber&isbright_1&isstar_2])
    f2b0=len(master_in[isfiber&isfaint_1&isbright_2])
    f2f0=len(master_in[isfiber&isfaint_1&isfaint_2])
    f2fh0=len(master_in[isfiber&isfaint_1&isfainthip_2])
    f2s0=len(master_in[isfiber&isfaint_1&isstar_2])
    fh2b0=len(master_in[isfiber&isfainthip_1&isbright_2])
    fh2f0=len(master_in[isfiber&isfainthip_1&isfaint_2])
    fh2fh0=len(master_in[isfiber&isfainthip_1&isfainthip_2])
    fh2s0=len(master_in[isfiber&isfainthip_1&isstar_2])
    s2b0=len(master_in[isfiber&isstar_1&isbright_2])
    s2f0=len(master_in[isfiber&isstar_1&isfaint_2])
    s2fh0=len(master_in[isfiber&isstar_1&isfainthip_2])
    s2s0=len(master_in[isfiber&isstar_1&isstar_2])
    b2b.append(b2b0)
    b2f.append(b2f0)
    b2fh.append(b2fh0)
    b2s.append(b2s0)
    f2b.append(f2b0)
    f2f.append(f2f0)
    f2fh.append(f2fh0)
    f2s.append(f2s0)
    fh2b.append(fh2b0)
    fh2f.append(fh2f0)
    fh2fh.append(fh2fh0)
    fh2s.append(fh2s0)
    s2b.append(s2b0)
    s2f.append(s2f0)
    s2fh.append(s2fh0)
    s2s.append(s2s0)
    fib.append(i)
b2b,b2f,b2fh,b2s=np.array(b2b),np.array(b2f),np.array(b2fh),np.array(b2s)
f2b,f2f,f2fh,f2s=np.array(f2b),np.array(f2f),np.array(f2fh),np.array(f2s)
fh2b,fh2f,fh2fh,fh2s=np.array(fh2b),np.array(fh2f),np.array(fh2fh),np.array(fh2s)
s2b,s2f,s2fh,s2s=np.array(s2b),np.array(s2f),np.array(s2fh),np.array(s2s)
fib=np.array(fib)

In [22]:
new_table=Table([fib,b2b,b2f,b2fh,b2s,f2b,f2f,f2fh,f2s,fh2b,fh2f,fh2fh,fh2s,s2b,s2f,s2fh,s2s],
               names=["fib","b2b","b2f","b2fh","b2s","f2b","f2f","f2fh","f2s","fh2b","fh2f","fh2fh","fh2s","s2b","s2f","s2fh","s2s"])
new_table

fib,b2b,b2f,b2fh,b2s,f2b,f2f,f2fh,f2s,fh2b,fh2f,fh2fh,fh2s,s2b,s2f,s2fh,s2s
int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
0,0,1,0,0,0,2,0,0,0,0,0,0,0,4,0,0
1,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,5
3,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,4
4,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,3
5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,7
6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,8
7,0,2,0,0,0,1,0,0,0,0,0,0,0,10,0,0
8,1,0,0,0,1,0,0,0,0,0,0,0,6,0,0,0


How many fibers

In [25]:
colnames=["b2b","b2f","b2fh","b2s","f2b","f2f","f2fh","f2s","fh2b","fh2f","fh2fh","fh2s","s2b","s2f","s2fh","s2s"]
names=["bright to bright","bright to faint","bright to faint_hip","bright to star","faint to bright","faint to faint","faint to faint_hip","faint to star",
       "faint_hip to bright","faint_hip to faint","faint_hip to faint_hip","faint_hip to star","star to bright","star to faint","star to faint_hip","star to star"]

for i in range(len(colnames)):
    print("Number of fibers",names[i],"=",len(new_table[new_table[colnames[i]]!=0]))

Number of fibers bright to bright = 531
Number of fibers bright to faint = 617
Number of fibers bright to faint_hip = 41
Number of fibers bright to star = 1617
Number of fibers faint to bright = 363
Number of fibers faint to faint = 833
Number of fibers faint to faint_hip = 24
Number of fibers faint to star = 1329
Number of fibers faint_hip to bright = 60
Number of fibers faint_hip to faint = 87
Number of fibers faint_hip to faint_hip = 48
Number of fibers faint_hip to star = 231
Number of fibers star to bright = 524
Number of fibers star to faint = 818
Number of fibers star to faint_hip = 47
Number of fibers star to star = 2939
